In [1]:
import gdown
import os

In [2]:
url="https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing"

In [3]:
file_id = url.split("/")[-2]

In [4]:
print(file_id)

1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3


In [5]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "kidney-CT-Scan-data.zip")

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=e13e7c19-79df-4f13-8bc1-7ba603cc5eee
To: /home/acer/Desktop/Projects/kidneyTumorDetection/research/kidney-CT-Scan-data.zip
100%|██████████| 57.7M/57.7M [03:48<00:00, 253kB/s]


'kidney-CT-Scan-data.zip'

In [7]:
%pwd

'/home/acer/Desktop/Projects/kidneyTumorDetection/research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'/home/acer/Desktop/Projects/kidneyTumorDetection'

In [12]:
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
from kidneyTumorDetection.constants import *
from kidneyTumorDetection.utils.common import read_yaml, create_directories

In [15]:
print(PARAMS_FILE_PATH)
print(CONFIG_FILE_PATH)

params.yaml
config/config.yaml


In [16]:
%pwd

'/home/acer/Desktop/Projects/kidneyTumorDetection'

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [27]:
import os
import zipfile
import gdown
from kidneyTumorDetection import logger
from kidneyTumorDetection.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-14 00:43:40,606: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-14 00:43:40,623: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-14 00:43:40,630: INFO: common: created directory at: artifacts]
[2024-09-14 00:43:40,637: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-14 00:43:40,644: INFO: 3320145291: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=3ad27539-0701-46e2-a25c-bd9ef631d037
To: /home/acer/Desktop/Projects/kidneyTumorDetection/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:09<00:00, 6.16MB/s]


[2024-09-14 00:43:55,473: INFO: 3320145291: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
